In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from ApartmentsDatasetPyTorch import ApartmentsDatasetPyTorch

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Create the dataset
dataset = ApartmentsDatasetPyTorch(
    data_dir = "../data/myrealty_apartments.csv",
    images_dir = '../images', 
    transform = transform
)

# Create the dataloader
dataloader = DataLoader(
    dataset,
    batch_size = 32,
    shuffle = True
)

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming you have uncommented and defined the transform, dataset, and dataloader

# Define a simple CNN model for regression
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 64 * 64, 500)  # Adjusted for the correct input size
        self.fc2 = nn.Linear(500, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)  # Adjust the reshaping here
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Instantiate the model

model = SimpleCNN()
try_gpu = False
if try_gpu:
    mps_device = torch.device("mps")
    model.to(mps_device)



# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, prices = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, prices.view(-1, 1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

Epoch 1, Loss: 47489989911.27273


KeyboardInterrupt: 